In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
import sklearn.metrics

In [93]:
def analysis(O_input, X_input, grouped):
    print("analysize ... {}, {}.csv".format(O_input, X_input))
    O = pd.read_csv("{}.csv".format(O_input))
    X = pd.read_csv("{}.csv".format(X_input))
   
    data = O.append(X)
    
    data.describe()
    #data
    if grouped == False:
        X=data[['성별코드','연령대코드(5세단위)','시력(좌)','시력(우)','청력(좌)','청력(우)','수축기혈압','이완기혈압','식전혈당(공복혈당)','총콜레스테롤','트리글리세라이드','혈색소','요단백','혈청크레아티닌','(혈청지오티)AST','(혈청지오티)ALT','감마지티피','흡연상태','음주여부','구강검진 수검여부','치석','whtr']]
    else:
        X=data[['성별코드','연령대코드(5세단위)','시력(좌)','시력(우)','청력(좌)','청력(우)','혈압','식전혈당(공복혈당)','총콜레스테롤','트리글리세라이드','혈색소','요단백','혈청크레아티닌','(혈청지오티)AST','(혈청지오티)ALT','감마지티피','흡연상태','음주여부','구강검진 수검여부','치석','whtr']]
    Y=data[['고지혈증']]

    #X
    X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size = 0.2)

    clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train, Y_train)
    #clf.predict(X_train)

    def Accuracy(label, pred):
        Acc = None
        correct = 0
        for i in range(len(label)):
            if label[i] == pred[i]:
                correct += 1
        Acc = correct / len(label)
        return Acc

    def Precision(label, pred):
        precision = None
        TP = 0
        FP = 0
        for i in range(len(label)):
            if pred[i] == 1:
                if label[i] == 1:
                    TP += 1
                else:
                    FP += 1
        precision = TP / (TP + FP)
        return precision

    def Recall(label, pred):
        recall = None
        TP = 0
        FN = 0
        for i in range(len(label)):
            if label[i] == 1:
                if pred[i] == 1:
                    TP += 1
                else:
                    FN += 1
        recall = TP / (TP + FN)
        return recall

    accuracy = Accuracy(Y_test['고지혈증'].values,clf.predict(X_test))

    sklearn.metrics.accuracy_score(Y_test['고지혈증'].values,clf.predict(X_test))

    precision = Precision(Y_test['고지혈증'].values,clf.predict(X_test))

    sklearn.metrics.precision_score(Y_test['고지혈증'].values,clf.predict(X_test))

    recall = Recall(Y_test['고지혈증'].values,clf.predict(X_test))

    sklearn.metrics.recall_score(Y_test['고지혈증'].values,clf.predict(X_test))

    def DCG(label, hypo, at=10):
        # ========================= EDIT HERE =========================
        dcg = None
        dcg = []
        temp_DCG = []
        temp_index = []

        #hypo 에 따라 정렬
        temp_index = sorted(range(len(hypo)), reverse=True, key=lambda k: hypo[k])
        temp_label = label[temp_index]

        for j in range(at):
            temp_DCG.append((1/np.log2(j+2)) * temp_label[j])
        #print("temp_DCG:", end="")
        #print(temp_DCG)

        dcg=sum(temp_DCG)
    # =============================================================
        return dcg

    def IDCG(label, hypo, at=10):
        # ========================= EDIT HERE =========================
        idcg = None
        idcg = []
        temp_IDCG = []

        # ideal 하게 정렬
        temp_label = sorted(label,reverse=True)

        for j in range(at):
            temp_IDCG.append((1 / np.log2(j + 2)) * temp_label[j])
        #print("temp_IDCG:", end="")
        #print(temp_IDCG)

        idcg=sum(temp_IDCG)
        # =============================================================
        return idcg

    dcg = DCG(Y_test['고지혈증'].values,clf.predict_proba(X_test)[:,0],at = 100)

    idcg = IDCG(Y_test['고지혈증'].values,clf.predict_proba(X_test)[:,0],at = 100)

    ndcg = dcg/idcg

    ndcg

    print("accuracy : {:.4f}".format(accuracy))
    print("precision : {:.4f}".format(precision))
    print("recall : {:.4f}".format(recall))
    print("ndcg : {:.4f}".format(ndcg))

In [96]:
analysis("5000_1","10000_0",False)

analysize ... 5000_1, 10000_0.csv


C:\Users\brigh\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy : 0.7433
precision : 0.6848
recall : 0.3986
ndcg : 0.1173


C:\Users\brigh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [97]:
analysis("g_5000_1","g_10000_0",True)

analysize ... g_5000_1, g_10000_0.csv


C:\Users\brigh\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy : 0.7447
precision : 0.6760
recall : 0.4194
ndcg : 0.0319


C:\Users\brigh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [99]:
def do_group(X_num, Y_num):
    analysis("{}_1".format(str(X_num)), "{}_0".format(str(Y_num)),False)
    analysis("g_{}_1".format(str(X_num)), "g_{}_0".format(str(Y_num)),True)    

In [167]:
do_group(10000,10000)

analysize ... 10000_1, 10000_0.csv


C:\Users\brigh\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\brigh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\brigh\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy : 0.7087
precision : 0.7278
recall : 0.6725
ndcg : 0.0995
analysize ... g_10000_1, g_10000_0.csv
accuracy : 0.7133
precision : 0.7236
recall : 0.6892
ndcg : 0.0157


C:\Users\brigh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
